In [0]:
# Installation cell
%%shell
if ! command -v julia 2>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.0/julia-1.0.5-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


In [0]:
using Pkg

pkg"add CuArrays"
pkg"add Flux"
pkg"add Plots"
pkg"add StatsBase"

pkg"precompile"

using Base.Iterators: partition

using CuArrays
using Flux
using Flux: onehot, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample

using LinearAlgebra
using Random
using Statistics

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed Requires ─────────── v0.5.2
 Installed CUDAapi ──────────── v2.0.0
 Installed GPUArrays ────────── v2.0.0
 Installed MacroTools ───────── v0.5.2
 Installed Adapt ────────────── v1.0.0
 Installed AbstractFFTs ─────── v0.4.1
 Installed CuArrays ─────────── v1.4.7
 Installed DataStructures ───── v0.17.6
 Installed CEnum ────────────── v0.2.0
 Installed TimerOutputs ─────── v0.5.3
 Installed CUDAnative ───────── v2.5.5
 Installed CUDAdrv ──────────── v4.0.4
 Installed LLVM ─────────────── v1.3.2
 Installed NNlib ────────────── v0.6.0
 Installed OrderedCollections ─ v1.1.0
  Updating `~/.julia/environments/v1.0/Project.toml`
  [3a865a2d] + CuArrays v1.4.7
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [621f4979] + AbstractFFTs v0.4.1
  [79e6a3ab] + Adapt v1.0.0
  [fa961155] + CEnum v0.2.0
  [3895d2a7] + CUDAapi v2.0.0
  

┌ Info: Precompiling StatsBase [2913bbd2-ae8a-5f71-8c99-4fb6c76f3a91]
└ @ Base loading.jl:1192


Precompiling CuArrays


┌ Info: Precompiling CuArrays [3a865a2d-5b23-5a0f-bc46-62713ec82fae]
└ @ Base loading.jl:1192


Precompiling Flux


┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1192


Precompiling Plots


┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1192
┌ Warning: Error requiring CuArrays from Flux:
│ LoadError: UndefVarError: libcudnn not defined
│ Stacktrace:
│  [1] getproperty(::Module, ::Symbol) at ./sysimg.jl:13
│  [2] top-level scope at none:0
│  [3] include at ./boot.jl:317 [inlined]
│  [4] include_relative(::Module, ::String) at ./loading.jl:1044
│  [5] include at ./sysimg.jl:29 [inlined]
│  [6] include(::String) at /root/.julia/packages/Flux/qXNjB/src/Flux.jl:1
│  [7] top-level scope at none:0
│  [8] eval at ./boot.jl:319 [inlined]
│  [9] eval at /root/.julia/packages/Flux/qXNjB/src/Flux.jl:1 [inlined]
│  [10] (::getfield(Flux, Symbol("##150#154")))() at /root/.julia/packages/Requires/9Jse8/src/require.jl:67
│  [11] err(::getfield(Flux, Symbol("##150#154")), ::Module, ::String) at /root/.julia/packages/Requires/9Jse8/src/require.jl:38
│  [12] #149 at /root/.julia/packages/Requires/9Jse8/src/require.jl:66 [inlined]
│  [13] withpath(::get

# A concrete realisation of a recurrent neural network #

## Housekeeping ##

### Assignment 2: “Half-bag” ###

Released after wrestling with Colaboratory, I had a piece of code that could bring down the whole runtime and I had no idea which piece of my code that did it…

Will the deadline work for you? Does it collide with other deadlines?

Please have a look at it in its entirery, even if you are not planning on working on it right now. After Thursday I will be slower to respond to feedback.

## Recap: recurrent neural networks ##

A differential state machine, that takes previous outputs and turns them into new outputs. 

$$ x_i \in \mathbb{R}^{d} $$

$$ h_i \in \mathbb{R}^{d} $$

$$ W \in \mathbb{R}^{d \times 2d} $$

$$ h_t = \textrm{tanh}(W[h_{t - 1}, x_t]) $$

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

[Image source](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Key point, at this stage we have not even mentioned a loss function. This is because we are not concerned with a specific loss, rather we want to turn a sequence of $t$ inputs into a state $h_t$ or into a sequence of outputs $\{x_1, \ldots, x_t\}$. A downstream network will later take either of these as input and we will backpropagate the error though our recurrent network (*“through time”* ) to accumulate a gradient with respect to its recurrent weights.

### Long Short-Term Memory ###

Introduced by [Hochreiter and Schmidhuber (1997)](https://www.bioinf.jku.at/publications/older/2604.pdf) and allows for longer dependencies.

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)


## Implementing a recurrent neural network ##



In [0]:
run(`curl -fsS https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt -o /tmp/shakespeare_input.txt`)

nothing

In [0]:
open("/tmp/shakespeare_input.txt") do f
    for _ in 1:32
        println(readline(f))
    end
end

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were


In [0]:
# Recurrent neural networks are slow to train, thus we need to downsample a bit.
text = collect(read("/tmp/shakespeare_input.txt", String))[1:100_000]

@show length(text) sort(unique(text)) length(sort(unique(text)))

nothing

length(text) = 100000
sort(unique(text)) = ['\n', ' ', '!', '&', '\'', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
length(sort(unique(text))) = 61


In [0]:
stopchar = '_'
@assert !(stopchar in unique(text))
outputs  = [stopchar, unique(text)...]
sort!(outputs)

nothing

In [0]:
enctext = map(c -> onehot(c, outputs), text)
encstop = onehot(stopchar, outputs)

nothing

Let us take a step back and reflect on what and why we have just done what we did…

In [0]:
onehot(outputs[1], outputs)

62-element Flux.OneHotVector:
  true
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false

In [0]:
onehot(outputs[2], outputs)

62-element Flux.OneHotVector:
 false
  true
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false

In [0]:
onehot(outputs[end], outputs)

62-element Flux.OneHotVector:
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
  true

We have encoded our $2^8$ space into a binary vector, recall the similarity here to old natural language processing methods we touched upon two weeks ago.

In [0]:
# These parameters need to be tuned to optimise speed.
maxinputlen = 64
numbatches  = 64

trainxs = collect(partition(batchseq(chunk(enctext, numbatches), encstop), maxinputlen))
trainys = collect(partition(batchseq(chunk(enctext[2:end], numbatches), encstop), maxinputlen))

nothing

Wait, I have been telling you that this model can handle variable-width inputs. Yet, we just capped everything at a given length, why?

In [0]:
# A “stacked” LSTM, with two layers.
model = Chain(
    LSTM(length(outputs), 128),
    LSTM(128, 128),
    Dense(128, length(outputs)),
    softmax) |> gpu

function loss(xs, ys)
    l = sum(crossentropy.(model.(gpu.(xs)), gpu.(ys)))
    # Resets the gradient tracking of the model, why?
    Flux.truncate!(model)
    l
end
# Increase the learning rate by 10x.
optimiser = ADAM(0.01)

nothing

In [0]:
@time for epoch in 1:128
    @time Flux.train!(loss, params(model), zip(trainxs, trainys), optimiser)
    println("$(epoch)\t$(loss(trainxs[1], trainys[1]))")
    flush(stdout)
end

 10.509655 seconds (14.73 M allocations: 689.053 MiB, 3.26% gc time)
1	122.35563f0 (tracked)
 10.551282 seconds (14.69 M allocations: 689.188 MiB, 5.09% gc time)
2	121.665245f0 (tracked)
 10.291102 seconds (14.69 M allocations: 688.898 MiB, 3.13% gc time)
3	121.26419f0 (tracked)
 10.279179 seconds (14.69 M allocations: 689.261 MiB, 3.08% gc time)
4	121.00234f0 (tracked)
 10.209422 seconds (14.69 M allocations: 689.929 MiB, 3.16% gc time)
5	120.35848f0 (tracked)
 10.217989 seconds (14.69 M allocations: 689.328 MiB, 3.22% gc time)
6	119.72657f0 (tracked)
 10.378828 seconds (14.69 M allocations: 689.379 MiB, 4.95% gc time)
7	119.71463f0 (tracked)
 10.247445 seconds (14.69 M allocations: 689.338 MiB, 3.07% gc time)
8	119.140305f0 (tracked)
 10.270134 seconds (14.69 M allocations: 688.892 MiB, 3.10% gc time)
9	118.65672f0 (tracked)
 10.208940 seconds (14.69 M allocations: 689.241 MiB, 3.10% gc time)
10	118.42187f0 (tracked)
 10.134122 seconds (14.69 M allocations: 688.883 MiB, 3.14% gc time

In [0]:
function sample(input, len)
    m = cpu(model)
    # Resets all internal states.
    Flux.reset!(m)
    buf = IOBuffer()
    # Prime the model with our initial input.
    for c in input[1:end - 1]
        m(onehot(c, outputs))
        write(buf, c)
    end
    write(buf, input[end])
    c = wsample(outputs, m(onehot(input[end], outputs)).data)
    write(buf, c)
    for i in 2:len
        c = wsample(outputs, m(onehot(c, outputs)).data)
        write(buf, c)
    end
    String(take!(buf))
end

println(sample("""
BORIS:
Next, I shall
""", 1024))

BORIS:
Next, I shall
Ent yourte Rongul hoep.

MENENIA:
You hio boeith of we cour themonk
the camm. Bus none
Ro callantes this. Comazend musend; To pnoo's, shoully the fuin_
To satt.
A kicd, wo frith u; to the ceaks
The cand, pure
That wikvine foe,
Wo raked
Thill nes, the yound; not buns; he'c-
Bou, afirt. CoRouns yourse,
Werow hick ald comes be ten mlice fore
'ticaraped, and, a par'd,-Yel, ne for shat:
The revas shiese,;
Aln beisd thowirnd: hace sendetsor, woult he ave wome

Merconde, will whoucd ughole amam, the the ie,
Thet wall.
I free. Your
I tiy whoce you hir th.

WICINIUS:
Why faund,,
Tars you
Wlat. at silk, an.

MENENIUTU
Tint he is.

SICINIUS
Se' yol tre?

CORIOLANUS:
Preate
Whey ore werlds. Hendes the thay pero:
Praply tres ge
-fil the.
Ho to buss an yer,
Sound pofind.
Your lade no now mog
Pulvers of frabe trlale wits?

MENENIUS:
Nowe thiul is prosendss,

Oy ulace cofguct,
O'd the inoug mingt
Af,: pillys: faved reark?

CiRIONUS:
Fir, he whang at,
And beny wis have senfert, I d

## Reminder to Pontus ##

After the lecture, go to Teaching & Learning to see what the status is for scheduling the “Neural methods for knowledge graphs” lecture.